In [1]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import sys
import os
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()
hf_token = os.getenv("HUGGINGFACE_API_KEY")
login(token= hf_token ) 

In [2]:
#! pip install sentence-transformers

In [3]:
os.getcwd()

'D:\\数字化0512\\葡萄牙黄金签证agent'

#### Split the doucment into Chunks & Store them in Vector Store
把 PDF 文件处理成「可搜索可调用」的向量数据库，供后续问答使用

In [4]:
loader = PyPDFLoader("./data/___Lei n.º 23_2007, de 04 de Julho.pdf")
loader 

In [5]:
def ingest():
    # Get the doc
    loader = PyPDFLoader("./data/___Lei n.º 23_2007, de 04 de Julho.pdf")
    pages = loader.load_and_split() ##加载PDF文档，并按照页自动分为pages，每页是一个document对象
    # Split the pages by char
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,## 把每页切成1024字符的段落，带100字重叠，目的是避免语义断裂
        chunk_overlap=100,
        length_function=len,
        add_start_index=True, ## 为了记录原始文本位置，可溯源
    )
    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} documents into {len(chunks)} chunks.")
    # 嵌入模型初始化，使用bge-m3模型嵌入，将文本块转换为向量表示。bge-m3模型支持多语查询
    #embedding = FastEmbedEmbeddings()
    embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
    #Create vector store 用向量化之后的chunks构建一个chroma数据库，并持久化在./sql_chroma_db路径下面，后续调用无需重新处理
    Chroma.from_documents(documents=chunks,  embedding=embedding, persist_directory="./goldenvisa_chroma_db")

In [6]:
# only run this once to generate vector store
ingest()

Split 166 documents into 714 chunks.


C:\Users\Peijin\AppData\Local\Temp\ipykernel_32948\211745222.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


##### Create a RAG chain that retreives relevent chunks and prepares a response

In [7]:
from langchain_community.chat_models import ChatOllama

# 指定模型名称，必须和你拉取的一致
llm = ChatOllama(model="llama3")

# 测试调用
response = llm.invoke("What is the capital of France?")
print(response.content)

C:\Users\Peijin\AppData\Local\Temp\ipykernel_32948\204113417.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3")


The capital of France is Paris.


In [10]:
#! pip install langchain==0.1.20

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.callbacks import StdOutCallbackHandler

In [6]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.embeddings import HuggingFaceEmbeddings

In [9]:

# 构造 Prompt 模板
def build_prompt():
    return PromptTemplate.from_template(
        """
        <s>[Instructions] You are a helpful assistant. Use only the context below to answer the question.
        If the answer is not in the context, say: "No context available for this question."
        [/Instructions]</s>

        <s>[Input] Question: {input}
        Context: {context}
        Answer: [/Input]</s>
        """
    )


# 加载向量数据库
def load_vector_store():
    embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
    return Chroma(
        persist_directory="./goldenvisa_chroma_db",
        embedding_function=embedding
    )


# 构建 Retriever（设置 k 和阈值）
def build_retriever(vector_store):
    return vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.5,
        },
    )


# 提取 source 信息
def format_answer_with_sources(answer: str, docs: list[Document]) -> str:
    sources = "\n".join([
        f"- Page: {doc.metadata.get('source', 'N/A')}"
        for doc in docs
    ])
    return f"{answer.strip()}\n\n📚 References:\n{sources}"


# 构建带输出 & source 的增强型 RAG chain
def rag_chain():
    # 1. Load LLM
    model = ChatOllama(model="llama3")

    # 2. Prompt
    prompt = build_prompt()

    # 3. Vector store
    vector_store = load_vector_store()

    # 4. Retriever
    retriever = build_retriever(vector_store)

    # 5. Stuff chain
    document_chain = create_stuff_documents_chain(model, prompt)

    # 6. Retrieval chain
    chain = create_retrieval_chain(retriever, document_chain)

    # 7. 包装为带来源输出的执行函数
    def run_with_sources(user_input: str):
        result = chain.invoke(
            {"input": user_input},
            config={"callbacks": [StdOutCallbackHandler()]}
        )
        # 输出中包括 retrieved 文档（用于引用）
        answer = result["answer"]
        docs = result["context"]
        return format_answer_with_sources(answer, docs)

    return run_with_sources


In [13]:
qa_chain = rag_chain()
response = qa_chain("Quais são os requisitos para obter o visto gold em Portugal através do reagrupamento familiar?")
print(response)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given




> Entering new RunnableSequence chain...


> Entering new RunnableAssign<context> chain...


> Entering new RunnableParallel<context> chain...


> Entering new RunnableSequence chain...


> Entering new RunnableLambda chain...

> Finished chain.


C:\Users\Peijin\miniconda3\envs\myenv\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.5
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


> Entering new RunnableAssign<answer> chain...


> Entering new RunnableParallel<answer> chain...


> Entering new RunnableSequence chain...


> Entering new RunnableAssign<context> chain...


> Entering new RunnableParallel<context> chain...


> Entering new RunnableLambda chain...

> Finished chain.

> Finished chain.

> Finished chain.


> Entering new PromptTemplate chain...

> Finished chain.


> Entering new StrOutputParser chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.
Question: Quais são os requisitos para obter o visto gold em Portugal através do reagrupamento familiar?

Context:
Answer: Os requisitos para obter o visto gold em Portugal através do reagrupamento familiar são:

* Ser um membro da família de um cidadão português ou de uma pessoa com direito a residência permanente em Portugal;
* Ter morado continuamente em Portugal pelo menos três anos consecutivos,